Profile plot description:
The figure was generated using the profile plot utility of deeptools. Both sample show a peak in ChIP signal and an immediate drop off at the TSS and low signal all the way until the TES as expected. This shows open chromatin at the TSS as expected for conditions with high TF activity

Methods:
Obtained reads were first put through QC analysis with FastQC. After inspecting QC metrics, reads were then trimmed using Trimmomatic before alignment with Bowtie2 to an index built from the human reference genome (GRCh38), also using Bowtie2. Reads were then sorted and indexed with samtools before samtools flagstat was used to generate metrics for alignment. At this point QC reports and log files were compiled using MultiQC. BigWig files were then generated using deeptools bamCoverage utility, with coverage summarized using deeptools bamCoverage and corrPlot utilities. Peakcalling with MACS3 was then performed on the sorted BAM files. The bedtools intersect utility was then used to generate a set of reproducible peaks by identifying peaks from both replicates that overlapped by 50% reciprocally befor again using bedtools intersect to filter out peaks overlapping blacklisted regions. Peak annotation was done using HOMER with default parameters. Bedtools computeMatrix and plotProfile utilities were then used to generate a profile plot from BigWig files. Lastly, the HOMER findMotifsGenome.pl script was used to find motifs in the filtered peaks. 

QC Report Summary:
Firstly, looking at the FastQC reports, GC content appears to follow a roughly normal distribution but differs slightly between INPUT and IP samples, as well as having what looks to be a small second peak in all distributions. Given that we expect different sequences between IP and INPUT, this isn't a red flag. However, FastQC does consider the distributions for the INNPUT samples to show an issue, which may need to be investigated further. Quality scores also look fine, with high quality across all samples. Duplication is higher in IP samples but again that is to be expected. One concern is the much lower number of sequences in the rep2 INPUT, with 10M instead of the ~30M in other samples. Flagstat results don't seem to show red flags either, with most reads from all samples passing QC checks and mapping. Trimmomatic also shows not many sequences were dropped, which is a good sign. 

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pybedtools
import re


In [ ]:
# read in peaks
peaks = pd.read_table("results/filtered_peaks.bed", header=None)

# set colnames for readability
peaks.columns = ["chrom", "chromStart", "chromEnd", "name", "score", "strand", "signalValue", "pValue", "qValue", "peak"]
peaks.head()

,chrom,chromStart,chromEnd,name,score,strand,signalValue,pValue,qValue,peak
0,GL000009.2,195221,195358,rep1_peak_216,28,.,2.69423,2.80547,0.720430,75
1,GL000194.1,155811,155924,rep1_peak_280,14,.,1.55919,1.47921,0.251322,116
2,GL000208.1,47204,47410,rep1_peak_496,33,.,2.96256,3.32397,0.942213,167
3,GL000216.2,29138,29276,rep1_peak_673,28,.,2.75165,2.81053,0.720430,118
4,GL000224.1,31792,31992,rep1_peak_1045,13,.,1.75131,1.38178,0.251322,136


In [19]:
# read in rnaseq res
rnaseq = pd.read_table("rnaseq_res.txt")

# filter by padj
pcut = 0.05
rnaseq_fltr = rnaseq[rnaseq['padj'] < 0.05]
rnaseq_fltr.columns = ['gene_name', 'transcript', 'log2FoldChange', 'padj']
rnaseq_fltr.head()

,gene_name,transcript,log2FoldChange,padj
7,RAD9A,"NM_001243224,NM_004584",-0.994436,4.514560e-06
8,TMEM70,"NM_001040613,NM_017866,NR_033334",-0.655475,1.825129e-03
12,FAM133B,"NM_001040057,NM_001288584,NM_152789,NR_109929",-0.521628,3.368896e-02
14,LRRC20,"NM_001278211,NM_001278212,NM_001278213,NM_0012...",0.646662,1.476895e-02
22,SLC7A5,NM_003486,-0.785875,7.129022e-09


In [14]:
# Create a database from gtf
gtf = pd.read_csv('/projectnb/bf528/materials/project-2-chipseq/refs/gencode.v45.primary_assembly.annotation.gtf', sep="\t", comment="#", header=None)
gtf.columns = ['chrom', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

# Filter for 'gene' entries
genes = gtf[gtf['feature'] == 'gene']

# Extract gene_name and gene_id
genes['gene_name'] = genes['attribute'].str.extract('gene_name "([^"]+)"')
genes['gene_id'] = genes['attribute'].str.extract('gene_id "([^"]+)"')
genes.head()

/scratch/ipykernel_3613278/4222830413.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genes['gene_name'] = genes['attribute'].str.extract('gene_name "([^"]+)"')
/scratch/ipykernel_3613278/4222830413.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genes['gene_id'] = genes['attribute'].str.extract('gene_id "([^"]+)"')


,chrom,source,feature,start,end,score,strand,frame,attribute,gene_name,gene_id
0,chr1,HAVANA,gene,11869,14409,.,+,.,"gene_id ""ENSG00000290825.1""; gene_type ""lncRNA...",DDX11L2,ENSG00000290825.1
5,chr1,HAVANA,gene,12010,13670,.,+,.,"gene_id ""ENSG00000223972.6""; gene_type ""transc...",DDX11L1,ENSG00000223972.6
13,chr1,HAVANA,gene,14696,24886,.,-,.,"gene_id ""ENSG00000227232.6""; gene_type ""unproc...",WASH7P,ENSG00000227232.6
25,chr1,ENSEMBL,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267.1""; gene_type ""miRNA""...",MIR6859-1,ENSG00000278267.1
28,chr1,HAVANA,gene,29554,31109,.,+,.,"gene_id ""ENSG00000243485.5""; gene_type ""lncRNA...",MIR1302-2HG,ENSG00000243485.5


In [20]:
# merge rnaseq and gtf
merged = rnaseq_fltr.merge(genes[['chrom', 'start', 'end', 'strand', 'gene_name']], on='gene_name', how='inner')
merged.head()

,gene_name,transcript,log2FoldChange,padj,chrom,start,end,strand
0,RAD9A,"NM_001243224,NM_004584",-0.994436,4.514560e-06,chr11,67317871,67398410,+
1,TMEM70,"NM_001040613,NM_017866,NR_033334",-0.655475,1.825129e-03,chr8,73972437,73982783,+
2,FAM133B,"NM_001040057,NM_001288584,NM_152789,NR_109929",-0.521628,3.368896e-02,chr7,92560758,92590393,-
3,LRRC20,"NM_001278211,NM_001278212,NM_001278213,NM_0012...",0.646662,1.476895e-02,chr10,70298970,70382650,-
4,SLC7A5,NM_003486,-0.785875,7.129022e-09,chr16,87830023,87869507,-


In [25]:
gene_bed = merged[['chrom', 'start', 'end', 'gene_name']]
gene_bed['start'] = gene_bed['start'].astype(int)
gene_bed['end'] = gene_bed['end'].astype(int)

peak_bed = peaks[['chrom', 'chromStart', 'chromEnd', 'name']]
peak_bed.columns = ['chrom', 'start', 'end', 'name']

gene_bt = pybedtools.BedTool.from_dataframe(gene_bed)
peak_bt = pybedtools.BedTool.from_dataframe(peak_bed)

overlaps = gene_bt.intersect(peak_bt, wa=True, wb=True)
overlap_df = overlaps.to_dataframe(names=[
    'gene_chrom', 'gene_start', 'gene_end', 'gene_name',
    'peak_chrom', 'peak_start', 'peak_end', 'peak_name'
])
final_df = overlap_df.merge(rnaseq_fltr, on='gene_name', how='left')
final_df.head()

/scratch/ipykernel_3613278/3656577109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_bed['start'] = gene_bed['start'].astype(int)
/scratch/ipykernel_3613278/3656577109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_bed['end'] = gene_bed['end'].astype(int)
***** WARNING: File /scratch/pybedtools.0baylfq7.tmp has inconsistent naming convention for record:
chr1	1649716	1649856	rep1_peak_7020

***** WARNING: File /scratch/pybedtools.0baylfq7.tmp has inconsistent naming convention for record:

,gene_chrom,gene_start,gene_end,gene_name,peak_chrom,peak_start,peak_end,peak_name,transcript,log2FoldChange,padj
0,chr11,67317871,67398410,RAD9A,chr11,67335729,67335891,rep1_peak_187956,"NM_001243224,NM_004584",-0.994436,0.000005
1,chr11,67317871,67398410,RAD9A,chr11,67383261,67383505,rep1_peak_187988,"NM_001243224,NM_004584",-0.994436,0.000005
2,chr10,121989163,122254545,TACC2,chr10,122158200,122158410,rep1_peak_154761,"NM_001291876,NM_001291877,NM_001291878,NM_0012...",-0.780570,0.000004
3,chr3,66378797,66501263,LRIG1,chr3,66465422,66465558,rep1_peak_725040,NM_015541,-0.441669,0.008332
4,chr2,15940550,15947007,MYCN,chr2,15945038,15945162,rep1_peak_536093,"NM_001293228,NM_001293231,NM_001293233",1.961676,0.000093
